# 语义搜索
1. 读取文档， 按照页来管理，Document（文档的页）,List[Dpocument]
2. 分割文档，按照文本段落来管理，Chunck（段落）,List[Chunck]
3. 向量化： 文本段向量映射，需要嵌入模型来辅助
4. 向量库：把多个文本段存储到向量库


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

from langchain_text_splitters import RecursiveCharacterTextSplitter

PDF_FILE_PATH = "../data/中华人民共和国刑法.pdf"

loader = PyPDFLoader(file_path=Path(PDF_FILE_PATH))
docs = loader.load()
# print(len(docs), docs[0].__dict__)


# 文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, add_start_index=True, chunk_overlap=200
)

all_splits = text_splitter.split_documents(docs)
# print(len(all_splits), all_splits[0].__dict__)

# 向量化：嵌入模型辅助
from langchain_ollama.embeddings import OllamaEmbeddings

embedder = OllamaEmbeddings(model="nomic-embed-text")
# vector_0 = embedder.embed_query(text=all_splits[0].page_content)
# print(len(vector_0), vector_0)

# 向量数据库存储
from langchain_chroma import Chroma

vector_db = Chroma(
    collection_name="china_law",
    persist_directory="../data/chroma_db",
    embedding_function=embedder,
)

ids = vector_db.add_documents(documents=all_splits)
print(len(ids), type(ids))


153 <class 'list'>
